In [40]:
import os 
import numpy as np
from sklearn.model_selection import train_test_split# !Figure out how to put seed 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix #! need to reserach this again, make notes
from sklearn.svm import SVC, SVR
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET # reads xml bounding box annotation, used for create labels matrix function 
#import cv2 # maybe need it later for visualization, Image for bounding box is good now, also matplot for simple(no box)
import matplotlib.pyplot as plt    # Not currently used 
import dill


In [2]:
"""Suggestion for testing: don't have to resize necesarily or even decrease more, keep base resolution, prob more accurate"""
def Create_Data_Matrix(directory):
    i=0
    for file in os.listdir(directory):
        filename = directory+'/'+file

        # Loads image, converts to grayscale and resizes it to a 300x300 image
        y = np.array(Image.open(filename).convert('RGB').resize((300,300)))

        # Resizes 300x300 image to 90,000x1 array
        col_y = y.ravel()[:,np.newaxis]

        # Saves
        if i==0:
            data = col_y
        else:
            data = np.hstack((data, col_y))

        # Plots image
        """
        plt.figure(figsize=(5,5))
        plt.imshow(y, cmap='gray')
        plt.xticks([]),plt.yticks([])
        plt.show();
        """
        i+=1
        
    return data

In [3]:
def Create_Labels_Matrix(directory):
    i=0
    for file in os.listdir(directory):
        filename = directory+'/'+file
        tree = ET.parse(filename)
        root = tree.getroot()
        sample_annotations = []
        for neighbor in root.iter('bndbox'):
            xmin = int(neighbor.find('xmin').text)
            ymin = int(neighbor.find('ymin').text)
            xmax = int(neighbor.find('xmax').text)
            ymax = int(neighbor.find('ymax').text)
            sample_annotations.append([xmin, ymin, xmax, ymax])
            if i==0:
                data = sample_annotations
            else:
                data = np.vstack((data, sample_annotations))
            break;
        i+=1
        
    return data

In [4]:
"""Creating the data matrix"""

#!! Suggestion: extra pictures later in dimmed light: this is critical for extra accuracy 

#using absolute paths in order to save OneDrive Space (don't want it to just fill with training pics)
bottleCap_pics=r"C:/Users/plani/Documents/Design2 Pics/cap/imgs"
bottleCap_labels=r"C:/Users/plani/Documents/Design2 Pics/cap/labels"

lighter_pics=r"C:/Users/plani/Documents/Design 1/ML repo/copy/MachineLearning/ML pics/Lighter"
lighter_labels=r"C:/Users/plani/Documents/Design2 Pics/lighter/labels"

emptyClass_pics = r"C:/Users/plani/Documents/Design2 Pics/empty"

bottleCap_array = Create_Data_Matrix(bottleCap_pics)
lighter_array = Create_Data_Matrix(lighter_pics)
emptyClass_array = Create_Data_Matrix(emptyClass_pics)

In [5]:
bottleCap_boxArray = Create_Labels_Matrix(bottleCap_labels)
lighter_boxArray = Create_Labels_Matrix(lighter_labels)

In [6]:
# 133 bottle cap images, each img rescaled to 300x300 and ravel() called = 90k. Since each pixel has 3 channels RGB, it is
# 90k * 3 = 270k

print("bottle cap array size:",bottleCap_array.shape)
print("lighter array size:",lighter_array.shape)
print("empty class array size",emptyClass_array.shape)

print("bottle cap label array size:",bottleCap_boxArray.shape)
print("lighter label array size:",lighter_boxArray.shape)

bottle cap array size: (270000, 133)
lighter array size: (270000, 276)
empty class array size (270000, 67)
bottle cap label array size: (133, 4)
lighter label array size: (276, 4)


In [7]:
"""X: Feature Matrix for Regression, t: TARGET MATRIX """
X = np.hstack((bottleCap_array, lighter_array)).T
X = X/255.

t = np.vstack((bottleCap_boxArray, lighter_boxArray))

X.shape, t.shape

((409, 270000), (409, 4))

In [13]:
# class_id used for stratified train/test split
class_id = np.hstack((np.ones(133), 2*np.ones(276))) 
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=42, stratify=class_id)

X_train.shape, X_test.shape

((327, 270000), (82, 270000))

In [19]:
"""Model Definition"""
model = SVR(kernel='rbf', C=1E9, epsilon=0.5)

In [77]:
"""SVR Model Training"""
# cannot do model.fit(x_train, t_train) because t_train is a multivariable
# output (4 coordinates)
multi_model = MultiOutputRegressor(model).fit(X_train, t_train)

In [78]:
"""STORE MODEL"""
# store the model using the 'dill library, dill apparently "serializes 
#  better than pickle" 
dill_filename = "SVM_regress.pkl"
with open(dill_filename, 'wb') as file:
    dill.dump(multi_model, file)

In [35]:
"""SVR Model Prediction"""
y_train = multi_model.predict(X_train)
y_test = multi_model.predict(X_test)

In [72]:
y_train_copy = y_train
y_train_copy

array([[2382.50011187,  592.50012481, 2608.50008751,  760.50004057],
       [2367.49998094, 1103.49986472, 2652.50009358, 1293.50004269],
       [1184.50038157, 2680.50015724, 1790.49984316, 3271.50012385],
       ...,
       [2184.49999519,  963.49970162, 2528.50026975, 1285.49996106],
       [ 849.50007641,  197.50032829, 1207.50001738,  380.50058386],
       [1316.50005307,  716.49971267, 1791.49970665,  950.49981945]])

In [74]:
col1 = np.array(t_test[0:12,1])[:,np.newaxis]
col2 = np.array(y_test[0:12,1])[:, np.newaxis]

cols = np.hstack((col1,col2))
cols

array([[1161.        ,  896.67350013],
       [2542.        , 1893.30221435],
       [  12.        ,  523.70663811],
       [ 694.        ,  785.6377801 ],
       [1746.        , 1559.37761782],
       [2219.        , 1642.82764506],
       [1227.        , 1346.07967776],
       [1373.        ,  946.02353528],
       [ 365.        , 1133.69659632],
       [1966.        , 1857.07681945],
       [ 168.        , 1097.36020976],
       [2104.        , 1960.25831169]])

In [76]:
"""SVR Test Performace, shows 300 pixels off"""
score_abs = mean_absolute_error(t_test, y_test, multioutput='raw_values')
score_abs



array([299.69346936, 300.13675252, 314.29503114, 379.13225026])

In [45]:
"""SVR Training Performance, only shows 0.5 pixel value discrepancy"""
score_abs = mean_absolute_error(t_train, y_train, multioutput='raw_values')
score_abs

array([0.50000734, 0.50000316, 0.50002107, 0.49866465])